In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import pandas as pd
from datasets import load_dataset
import tqdm as tqdm

regression_dataset_tmdb_raw_df = load_dataset("ada-datadruids/regression_dataset_tmdb")['train'].to_pandas()
# interpret release_date as datetime
regression_dataset_tmdb_raw_df['release_date'] = pd.to_datetime(regression_dataset_tmdb_raw_df['release_date'])

# only take movies that are after 1920
regression_dataset_tmdb_raw_df = regression_dataset_tmdb_raw_df[regression_dataset_tmdb_raw_df['release_date'] > '1920-01-01']
regression_dataset_tmdb_raw_df = regression_dataset_tmdb_raw_df[regression_dataset_tmdb_raw_df['release_date'] < '2013-01-01']

regression_dataset_tmdb_raw_df.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords,based_on_book
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc...",False
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f...",False
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ...",False
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com...",False
5,550,Fight Club,8.438,27238,Released,1999-10-15,100853753,139,False,/hZkgoQYus5vegHoetLkCJzb17zJ.jpg,...,A ticking-time-bomb insomniac and a slippery s...,69.498,/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg,Mischief. Mayhem. Soap.,Drama,"Regency Enterprises, Fox 2000 Pictures, Taurus...",United States of America,English,"dual identity, rage and hate, based on novel o...",True


# Feature Evolution Over Time

## Popularity over time (only movies based on books)

In [24]:
based_on_books_df = regression_dataset_tmdb_raw_df[regression_dataset_tmdb_raw_df['based_on_book'] == 1]
not_based_on_books_df = regression_dataset_tmdb_raw_df[regression_dataset_tmdb_raw_df['based_on_book'] == 0]

based_on_books_df['num_languages'] = based_on_books_df['spoken_languages'].apply(
    lambda x: 1 if x is None else len(x.split(',')))
based_on_books_df['num_languages'] = based_on_books_df['num_languages'].replace(0, 1)

not_based_on_books_df['num_languages'] = not_based_on_books_df['spoken_languages'].apply(
    lambda x: 1 if x is None else len(x.split(',')))
not_based_on_books_df['num_languages'] = not_based_on_books_df['num_languages'].replace(0, 1)

display(based_on_books_df.head())
display(not_based_on_books_df.head())

C:\Users\LucaS\AppData\Local\Temp\ipykernel_29384\2146280726.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\LucaS\AppData\Local\Temp\ipykernel_29384\2146280726.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\LucaS\AppData\Local\Temp\ipykernel_29384\2146280726.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords,based_on_book,num_languages
5,550,Fight Club,8.438,27238,Released,1999-10-15,100853753,139,False,/hZkgoQYus5vegHoetLkCJzb17zJ.jpg,...,69.498,/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg,Mischief. Mayhem. Soap.,Drama,"Regency Enterprises, Fox 2000 Pictures, Taurus...",United States of America,English,"dual identity, rage and hate, based on novel o...",True,1
8,13,Forrest Gump,8.477,25409,Released,1994-06-23,677387716,142,False,/qdIMHd4sEfJSckfVJfKQvisL02a.jpg,...,92.693,/arw2vcBveWOVZr6pxd9XTd1TdQa.jpg,The world will never be the same once you've s...,"Comedy, Drama, Romance","Paramount, The Steve Tisch Company, Wendy Fine...",United States of America,English,"vietnam war, vietnam veteran, mentally disable...",True,1
9,671,Harry Potter and the Philosopher's Stone,7.916,25379,Released,2001-11-16,976475550,152,False,/hziiv14OpD73u9gAak4XDDfBKa2.jpg,...,185.482,/wuMc08IPKEatf9rnMNXvIDxqP4W.jpg,Let the magic begin.,"Adventure, Fantasy","Warner Bros. Pictures, Heyday Films, 1492 Pict...","United Kingdom, United States of America",English,"witch, school friend, friendship, london, engl...",True,1
15,120,The Lord of the Rings: The Fellowship of the Ring,8.402,23323,Released,2001-12-18,871368364,179,False,/x2RS3uTcsJJ9IfjNPcgDmukoEcQ.jpg,...,87.037,/6oom5QYQ2yQTMJIbnvbkBL9cHo6.jpg,One ring to rule them all,"Adventure, Fantasy, Action","New Line Cinema, WingNut Films, The Saul Zaent...","New Zealand, United States of America",English,"based on novel or book, orcs, elves, dwarf, ma...",True,1
16,122,The Lord of the Rings: The Return of the King,8.474,22334,Released,2003-12-01,1118888979,201,False,/2u7zbn8EudG6kLlBzUYqP8RyFU4.jpg,...,99.276,/rCzpDGLbOoPwLjy3OAm5NUPOTrC.jpg,The eye of the enemy is moving.,"Adventure, Fantasy, Action","New Line Cinema, WingNut Films, The Saul Zaent...","New Zealand, United States of America",English,"based on novel or book, orcs, elves, dwarf, br...",True,1


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords,based_on_book,num_languages
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc...",False,4
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f...",False,2
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ...",False,2
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com...",False,3
7,680,Pulp Fiction,8.488,25893,Released,1994-09-10,213900000,154,False,/suaEOtk1N1sgg2MTM7oZd2cfVp3.jpg,...,74.862,/d5iIlFn5s0ImszYzBPb8JPIfbXD.jpg,Just because you are a character doesn't mean ...,"Thriller, Crime","Miramax, A Band Apart, Jersey Films",United States of America,"English, Spanish, French","drug dealer, boxer, massage, stolen money, bri...",False,3


In [25]:
from src.utils.features_over_time_utils import plot_box_with_trends_over_time


plot_box_with_trends_over_time(
    based_on_books_df,
    value_column='popularity',
    title='Box Plot of Popularity of Book-Based Movies per 3-Year Interval',
    yaxis_label='Popularity',
    yaxis_range=(0, 65),
)

plot_box_with_trends_over_time(
    not_based_on_books_df,
    value_column='popularity',
    title='Box Plot of Popularity of Non-Book-Based Movies per 3-Year Interval',
    yaxis_label='Popularity',
    yaxis_range=(0, 65),
)

print("\n\n")

plot_box_with_trends_over_time(
    based_on_books_df,
    value_column='popularity',
    title='Box Plot of Popularity of Book-Based Movies per 3-Year Interval',
    yaxis_label='Popularity',
    use_log_y=True,
)

plot_box_with_trends_over_time(
    not_based_on_books_df,
    value_column='popularity',
    title='Box Plot of Popularity of Non-Book-Based Movies per 3-Year Interval',
    yaxis_label='Popularity',
    use_log_y=True,
)

## Budget over time

In [38]:
plot_box_with_trends_over_time(
    based_on_books_df,
    value_column='budget',
    title='Box Plot of Budget of Book-Based Movies per 3-Year Interval',
    yaxis_label='Budget',
    yaxis_range=(0, 1e8),
)

plot_box_with_trends_over_time(
    not_based_on_books_df,
    value_column='budget',
    title='Box Plot of Budget of Non-Book-Based Movies per 3-Year Interval',
    yaxis_label='Budget',
    yaxis_range=(0, 1e8),
)

## Revenue over time

In [42]:
plot_box_with_trends_over_time(
    based_on_books_df,
    value_column='revenue',
    title='Box Plot of Revenue of Book-Based Movies per 3-Year Interval',
    yaxis_label='Revenue',
    yaxis_range=(0, 3e8),
)

plot_box_with_trends_over_time(
    not_based_on_books_df,
    value_column='revenue',
    title='Box Plot of Revenue of Non-Book-Based Movies per 3-Year Interval',
    yaxis_label='Revenue',
    yaxis_range=(0, 3e8),
)

## Number of movies based on books over time

In [26]:
from src.utils.features_over_time_utils import plot_movie_count_over_time

plot_movie_count_over_time(
    based_on_books_df,
    release_date_column='release_date',
    title='Number of Movies Based on Books per 3-Year Interval'
)

plot_movie_count_over_time(
    not_based_on_books_df,
    release_date_column='release_date',
    title='Number of Movies Not Based on Books per 3-Year Interval'
)

## Number of languages per movie over time

In [27]:


plot_box_with_trends_over_time(
    based_on_books_df,
    value_column='num_languages',
    title='Box Plot of Number of Languages per Book-Based Movie per 3-Year Interval',
    yaxis_label='Number of Languages',
)

plot_box_with_trends_over_time(
    not_based_on_books_df,
    value_column='num_languages',
    title='Box Plot of Number of Languages per Non-Book-Based Movie per 3-Year Interval',
    yaxis_label='Number of Languages',
)

C:\Users\LucaS\anaconda3\envs\ada-project\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide



---

In [28]:
summaries_df = load_dataset("ada-datadruids/bookfilm_summaries_with_similarity_and_sentiment")['train'].to_pandas()

summaries_df.head()

,movie_id,book_title,movie_name,book_summary,film_summary,similarity,film_sentiment,film_sentiment_score,book_sentiment,book_sentiment_score
0,196176,The Siege of Trencher's Farm,Straw Dogs,"George Magruder, an American professor of Eng...","David Sumner , a timid American mathematician,...",0.451876,positive,0.333608,positive,0.994480
1,6703617,Journey to the West,The Forbidden Kingdom,The novel has 100 chapters. These can be divi...,"In this film, which is based loosely on the an...",0.559594,positive,0.995482,positive,0.995404
2,24416479,The Beast Master,Beastmaster 2: Through the Portal of Time,"It tells the story of Hosteen Storm, an ex-so...","Dar, the Beastmaster , is back and now he has ...",0.344548,positive,0.997562,positive,0.998812
3,9384481,The Blessing,Count Your Blessings,It is set in the post-war World War II period...,While visiting Grace Allingham in wartime Lond...,0.669707,positive,0.996070,positive,0.996607
4,22224559,The Last Song,The Last Song,Veronica “Ronnie” Miller’s life was turned up...,"At seventeen, Veronica ""Ronnie"" Miller remain...",0.852735,positive,0.978184,positive,0.998164


In [29]:
final_df = pd.read_csv("./../data/final_dataset.csv")

print(len(final_df))
#only take movies that are after 1920 and before 2013 because there is not enough data for the other years
final_df = final_df[final_df['release_date'] > '1920-01-01']
final_df = final_df[final_df['release_date'] < '2013-01-01']

# interpret movie_id as integer in summaries_df and merge on movie_id with final_df
summaries_df['movie_id'] = summaries_df['movie_id'].astype(int)
book_based_final_df = final_df.merge(summaries_df, on='movie_id', how='inner')

print("book_based length:", len(book_based_final_df))
book_based_final_df.head()

1845
book_based length: 829


,Unnamed: 0,movie_id,movie_cluster,movie_name_x,release_date,runtime_x,language,country,genres,movie_year,...,review_count,book_title,movie_name_y,book_summary,film_summary,similarity,film_sentiment,film_sentiment_score,book_sentiment,book_sentiment_score
0,5,196176,/m/01bwgr,Straw Dogs,1971-11-03,118.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/01jfsb"": ""Thriller"", ""/m/07s9rl0"": ""Drama""}",1971,...,54.0,The Siege of Trencher's Farm,Straw Dogs,"George Magruder, an American professor of Eng...","David Sumner , a timid American mathematician,...",0.451876,positive,0.333608,positive,0.994480
1,6,6703617,/m/0gj6pd,The Forbidden Kingdom,2008-04-04,105.0,"{""/m/0653m"": ""Standard Mandarin"", ""/m/02h40lc""...","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/08322"": ""Wuxia"", ""/m/0hj3n2k"": ""Fantasy A...",2008,...,420.0,Journey to the West,The Forbidden Kingdom,The novel has 100 chapters. These can be divi...,"In this film, which is based loosely on the an...",0.559594,positive,0.995482,positive,0.995404
2,7,24416479,/m/07s7_c0,Beastmaster 2: Through the Portal of Time,1991-08-31,107.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0dz8b"": ""Sword and sorcery"", ""/m/06n90"": ...",1991,...,68.0,The Beast Master,Beastmaster 2: Through the Portal of Time,"It tells the story of Hosteen Storm, an ex-so...","Dar, the Beastmaster , is back and now he has ...",0.344548,positive,0.997562,positive,0.998812
3,8,9384481,/m/0286qw2,Count Your Blessings,1959,102.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama"", ""/m/01z4y"": ""Comedy""}",1959,...,127.0,The Blessing,Count Your Blessings,It is set in the post-war World War II period...,While visiting Grace Allingham in wartime Lond...,0.669707,positive,0.996070,positive,0.996607
4,14,22224559,/m/07ykkx5,The Last Song,2010-03-31,107.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01j1n2"": ""Coming of age"", ""/m/07s9rl0"": ""...",2010,...,11534.0,The Last Song,The Last Song,Veronica “Ronnie” Miller’s life was turned up...,"At seventeen, Veronica ""Ronnie"" Miller remain...",0.852735,positive,0.978184,positive,0.998164


## 'normalized_rating_x' over time

In [30]:
plot_box_with_trends_over_time(
    book_based_final_df,
    value_column='normalized_rating_x',
    title='Box Plot of Normalized Rating x of Book-Based Movies per 3-Year Interval',
    yaxis_label='Normalized Rating',
    yaxis_range=(0, 1),
)

## 'normalized_rating_y' over time

In [31]:
plot_box_with_trends_over_time(
    book_based_final_df,
    value_column='normalized_rating_y',
    title='Box Plot of Normalized Rating y of Books per 3-Year Interval',
    yaxis_label='Normalized Rating',
    yaxis_range=(0.3, 0.9),
)

## 'similarity' feature over time

In [32]:
plot_box_with_trends_over_time(
    book_based_final_df,
    value_column='similarity',
    title='Box Plot of Book-Movie Summary Similarity per 3-Year Interval',
    yaxis_label='Summary Embedding Cosine Similarity',
)

## 'sentiment' feature difference over time

In [33]:
# as float
book_based_final_df['film_sentiment_score'] = book_based_final_df['film_sentiment_score'].astype(float)
book_based_final_df['book_sentiment_score'] = book_based_final_df['book_sentiment_score'].astype(float)
book_based_final_df['sentiment_difference_movie_book'] = book_based_final_df['film_sentiment_score'] - book_based_final_df['book_sentiment_score']

book_based_final_df['sentiment_difference_movie_book'].describe()

count    829.000000
mean      -0.104786
std        0.644145
min       -1.993367
25%       -0.289051
50%       -0.001589
75%        0.015245
max        1.994675
Name: sentiment_difference_movie_book, dtype: float64

In [34]:
plot_box_with_trends_over_time(
    book_based_final_df,
    value_column='sentiment_difference_movie_book',
    title='Box Plot of Book-Movie Summary Sentiment Difference per 3-Year Interval',
    yaxis_label='Summary Sentiment Difference',
)